In [ ]:
import json
import requests
import boto3
import csv
from io import StringIO

def csv_string_to_dict(csv_string):
    result = []
    csv_reader = csv.DictReader(StringIO(csv_string))
    for row in csv_reader:
        result.append(row)
    return result
    
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

dynamodb = boto3.client('dynamodb')
s3_client = boto3.client('s3')

credits = ''
keywords = ''
links = ''
links_small = ''
movies_metadata = ''
ratings_small = ''

def lambda_handler(event, context):

    body = json.loads(event['body'])
    table_name = body['table_name']
    print(table_name)
    TODO implement
    global credits
    global keywords
    global links
    global links_small
    global movies_metadata
    global ratings_small
    
    if(credits === ''):
        credits = getFileFromS3('credits.csv')
        
    if(keywords === ''):
        keywords = getFileFromS3('keywords.csv')
        
    if(links === ''):
        links = getFileFromS3('links.csv')
        
    if(links_small === ''):
        links_small = getFileFromS3('links_small.csv')
        
    if(movies_metadata == ''):
        movies_metadata = getFileFromS3('movies_metadata.csv')
        insertData(movies_metadata, 'movies_metadata')
        
    if(ratings_small === ''):
        ratings_small = getFileFromS3('ratings_small.csv')
        
    response = getFileFromS3('movies_metadata.csv')
    
    response = getData('keywords')

    print(response)
    insertData()
    
    return {
        'statusCode': 200,
        'body': json.dumps(response)
    }


def getData(table_name):
    scan_params = {
        'TableName': table_name
    }
    
    response = dynamodb.scan(**scan_params)
    
    finalArr = []
    
    while True:
        items = response['Items']
        for item in items:
            finalArr.append(item)
            
        if 'LastEvaluatedKey' in response:
            scan_params['ExclusiveStartKey'] = response['LastEvaluatedKey']
            response = dynamodb.scan(**scan_params)
        else:
            break

    return finalArr

def insertData(data, table_name):

    for row in data:
        item_dict = {}
        for key, value in row.items():
            temp_dict = {}
            if value != None and (value.lower() == 'true' or value.lower() == 'false'):
                temp_dict['B'] = value
                item_dict[key] = temp_dict
            elif value != None and value.isdigit():
                temp_dict['N'] = value
                item_dict[key] = temp_dict
            elif value != None and is_float(value):
                temp_dict['N'] = value
                item_dict[key] = temp_dict
            elif value != None and value.startswith('[') and value.endswith(']'):
                temp_dict['S'] = value
                item_dict[key] = temp_dict
            else:
                temp_dict['S'] = value
                item_dict[key] = temp_dict

        print(item_dict)
        dynamodb.put_item(
            TableName=table_name,
            Item=item_dict
        )


def getFileFromS3(file_name):
    bucket_name = 'movierecommendationsystem'
    s3_response = s3_client.get_object(Bucket=bucket_name, Key=file_name)
    file_data = s3_response ["Body"].read().decode('utf')
    return csv_string_to_dict(file_data)
    